# The Movies Dataset TF-IDF Content-Based Recommendation

##Понимание данных:

### Набор данных

Дан набор данных, содержащий информацию о 45,000 фильмах, выпущенных до июля 2017 года. Набор данных представлен на ресурсе Kaggle по ссылке https://www.kaggle.com/rounakbanik/the-movies-dataset
где представлено следующее описание составляющих файлов (выполнен перевод на русский язык):

`movies_metadata.csv:` Основной файл метаданных фильмов. Содержит информацию о 45 000 фильмов, представленных в наборе данных Full MovieLens. В таблицы представлены плакаты, фоны, бюджет, доход, даты выпуска, языки, страны-производители и компании.

`keywords.csv:` Содержит ключевые слова сюжета для наших фильмов MovieLens. Доступен в виде строкового объекта JSON.

`credits.csv:` Состоит из информации об актерах и съемках всех наших фильмов. Доступен в виде строкового объекта JSON. 

`links.csv:` Файл содержит идентификаторы TMDB и IMDB всех фильмов, представленных в наборе данных Full MovieLens.

`links_small.csv:` Содержит идентификаторы TMDB и IMDB небольшого подмножества из 9000 фильмов полного набора данных.

`ratings_small.csv:` Подмножество 100 000 оценок от 700 пользователей на 9 000 фильмов.

`ratings.csv `- файл, содержащий полный список оценок, выставленных пользователями фильмам



Рассмотрим подробнее две таблицы:

*   `movies_metadata.csv `
*   `ratings.csv `

In [1]:
import pandas as pd
import numpy as np

In [3]:
! pip install --upgrade --no-cache-dir gdown

In [4]:
!gdown 18PC17_iR9f4WuGIKLSHU3drPt1jdCofA

Downloading...
From: https://drive.google.com/uc?id=18PC17_iR9f4WuGIKLSHU3drPt1jdCofA
To: /Users/vaceslavefimov/Documents/ITMO/Data Analysis/LW3/ratings.csv
100%|████████████████████████████████████████| 710M/710M [01:33<00:00, 7.58MB/s]


In [5]:
!gdown 1wTPUbZx4C2zhdqzPk3yxKyVZJIxQcsvy

Downloading...
From: https://drive.google.com/uc?id=1wTPUbZx4C2zhdqzPk3yxKyVZJIxQcsvy
To: /Users/vaceslavefimov/Documents/ITMO/Data Analysis/LW3/movies_metadata.csv
100%|██████████████████████████████████████| 34.4M/34.4M [00:07<00:00, 4.73MB/s]


Начинаем с анализа датафрейма movies_metadata

In [13]:
metadata = pd.read_csv("movies_metadata.csv")
print(f'Всего строк: {len(metadata)}')

Всего строк: 45466


## Задание 1

Удадите из датафрейма metadata строки, в которых отсутствует описание. Обратите внимание, что у некоторых фильмов формально описание есть, но там написано No overview found, No overview, No movie overview available, Released. Такие строки тоже нужно удалить. 

In [15]:
values_to_drop = ('No overview found', 'No overview', 'No movie overview available', 'Released')
metadata = metadata[(~metadata.overview.isin(values_to_drop)) & (pd.notnull(metadata.overview))].reset_index(drop=True)
print(f'Всего строк: {len(metadata)}')

Всего строк: 44507


## Задание 2

Оставьте в датафрейме столбцы `'id', 'imdb_id', 'overview', 'title'`. Выведите 10 случайных строк.

In [16]:
metadata = metadata[['id', 'imdb_id', 'overview', 'title']]
metadata.sample(10)

,id,imdb_id,overview,title
37273,68016,tt1821317,"The movie is set in the backdrop of a town, wh...",Aadukalam
37017,370902,tt3579808,"In Yekaterinburg over the Christmas period, sh...",The Land of OZ
43610,353406,tt4636194,In this gleefully anarchic tour-de-force of pu...,Manoman
13976,14940,tt1176251,Young mattress salesman Brian decides to adopt...,Gigantic
33581,4893,tt0059905,No overview found.,Where the Spies Are
8890,41959,tt0095389,A young doctor is suspected when a series of J...,Jack's Back
3590,592,tt0071360,Surveillance expert Harry Caul (Gene Hackman) ...,The Conversation
43086,51299,tt1828232,Life Cycles tells a spectacular story of the b...,Life Cycles
43,577,tt0114681,Susan wants to work in television and will the...,To Die For
4181,12639,tt0060897,Chico one of the remaining members of The Magn...,Return of the Seven


## Задание 3

In [ ]:
ratings = pd.read_csv('ratings.csv')
print(f'Всего строк: {len(ratings)}')

In [ ]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Убедимся в отсутствии пропусков

In [ ]:
pd.isnull(ratings).sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Объедините датафреймы `metadata` и `ratings` в один. Обратите внимание, что `'id'` в  `metadata`, этот тот же самый идентификатор, что и `'movie_id'` и `ratings`. Объединять нужно по этому идентификатору (также обратите внимание на его тип данных).

In [23]:
metadata['id'] = metadata['id'].astype(np.int64)

In [33]:
df = metadata.merge(ratings, left_on='id', right_on='movieId')
df

,id,imdb_id,overview,title,userId,movieId,rating,timestamp
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,1923,862,3.0,858335006
1,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,2103,862,5.0,946044912
2,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,5380,862,1.0,878941641
3,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,6177,862,4.0,859415226
4,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,6525,862,4.0,857388995
...,...,...,...,...,...,...,...,...
11396975,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,33940,111109,2.5,1405878785
11396976,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,172224,111109,3.0,1399502972
11396977,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,210792,111109,3.0,1467090449
11396978,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,225396,111109,3.5,1399302912


## Задание 4

Появились ли пропуски в получившемся после объединения датафрейме? Если появились, то ответьте на вопрос "почему?" и удалите строки с пропусками.

Выведите размеры получившегося датафрейма и 10 случайных записей в нём.

In [34]:
df.isnull().sum()

id            0
imdb_id      47
overview      0
title         0
userId        0
movieId       0
rating        0
timestamp     0
dtype: int64

Заметим, что в столбце `'imdb_id'` появилось 47 пропусков. Можно предположить, что причиной этого стало наличие пропусков в исходном датасете в столбце `'imdb_id'`. При процедуре объединения двух датасетов могут мерджиться строки, как раз содержащие пропуски, что и является следствием их возникновения в новом датасете. Убедимся, что в metadata действительно имеются пропуски в столбце `'imdb_id'`:

In [35]:
metadata.isnull().sum()

id           0
imdb_id     15
overview     0
title        3
dtype: int64

Видно, что в искомом столбце содержится 15 пропусков. Наше предположение подтвердилось. Удалим эти строки:

In [36]:
print(f'Размер датафрейма до удаления: {df.shape}')
df = df[pd.notnull(df.imdb_id)]
print(f'Размер датафрейма после удаления: {df.shape}')

Размер датафрейма до удаления: (11396980, 8)
Размер датафрейма после удаления: (11396933, 8)


In [38]:
# выведем 10 случайных строк

df.sample(10)

,id,imdb_id,overview,title,userId,movieId,rating,timestamp
4807278,150,tt0083511,A hard-nosed cop reluctantly teams up with a w...,48 Hrs.,249517,150,4.5,1451760764
10246798,54272,tt0164984,Blind swordsman/masseuse Zatoichi befriends a ...,Adventures of Zatôichi,258194,54272,5.0,1278338893
7251347,344,tt0135790,Bank robber Kelle Grabowski escapes from priso...,"Bang, Boom, Bang",100944,344,3.0,1028363817
1993630,95,tt0120591,When an asteroid threatens to collide with Ear...,Armageddon,151665,95,3.5,1077558845
10841936,457,tt0048624,"The young Bavarian princess Elisabeth, who all...",Sissi,39393,457,4.0,839006401
9099949,2011,tt0808417,"In 1970s Iran, Marjane 'Marji' Statrapi watche...",Persepolis,257993,2011,3.5,1485526629
8310927,1381,tt0414993,"Spanning over one thousand years, and three pa...",The Fountain,242114,1381,1.0,948379399
6245720,587,tt0319061,Throughout his life Edward Bloom has always be...,Big Fish,148498,587,3.0,834316333
10642848,46578,tt0078960,A busload containing three cheerleading teams ...,Cheerleaders' Wild Weekend,74105,46578,4.0,1285186178
537589,2088,tt0107983,A corrupt cop gets in over his head when he tr...,Romeo Is Bleeding,257965,2088,4.5,1094627438


## Задание 5

Возьмите случайного пользователя (проверьте, чтобы у него было достаточное количество оценок). Сформируйте для этого пользователя рекомендацию методом коллаборативной фильтрации. Оцените качество это рекомендации.


Повторите эти рассчеты для большого числа пользователей и дайте интегральную оценку.

In [45]:
user_id = 107720
user_df = df.query(f'userId == {user_id}')
user_df

,id,imdb_id,overview,title,userId,movieId,rating,timestamp
154,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,107720,862,2.0,1030400148
1427,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",Heat,107720,949,4.0,1029864076
3379,710,tt0113189,James Bond must unmask the mysterious head of ...,GoldenEye,107720,710,2.0,1030409961
9709,1408,tt0112760,"Morgan Adams and her slave, William Shaw, are ...",Cutthroat Island,107720,1408,3.0,1029790905
21769,524,tt0112641,The life of the gambling paradise – Las Vegas ...,Casino,107720,524,3.0,1029866021
...,...,...,...,...,...,...,...,...
11380824,2286,tt0381111,Dolf a 15 year old boy is sent back in time by...,Crusade in Jeans,107720,2286,2.0,1030410627
11383223,3178,tt0211941,The story of writer William Seward Burroughs a...,Beat,107720,3178,3.0,1029792928
11386919,2331,tt0199232,The film is notable for presenting a more huma...,Jesus,107720,2331,3.0,1029793172
11391625,3104,tt0061683,A deformed tormented girl drowns herself after...,Frankenstein Created Woman,107720,3104,3.0,1029791297


In [47]:
from scipy.stats import pearsonr
from math import isnan

In [120]:
class CollaborativeFiltering:
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.users = set(df.userId)
    
    def common_movies(self, user1: int, user2: int) -> pd.DataFrame:
        columns = ['userId', 'movieId', 'rating']
        return pd.merge(
            self.df.query(f'userId == {user1}')[columns],
            self.df.query(f'userId == {user2}')[columns],
            on='movieId',
            suffixes=['1', '2']
        ).drop(columns={'userId1', 'userId2'})
    
    def euclid_distance(self, x: list, y: list) -> float:
        distance = 0
        for x_value, y_value in zip(x, y):
            distance += (x_value - y_value) ** 2
        return np.sqrt(distance)
    
    def similarity_distance(self, common_movies_df: pd.DataFrame) -> float:
        if len(common_movies_df) == 0:
            return 0
        return 1 / (1 + self.euclid_distance(common_movies_df['rating1'], common_movies_df['rating2']))
    
    def top_matches(self, user: int, n: int=10) -> pd.DataFrame:
        users = list(self.users - set([user]))
        users = users[:100]
        return pd.DataFrame(data={
            'user': users,
            'similarity': [self.similarity_distance(self.common_movies(user, cur_user)) for cur_user in users]
        }).sort_values(by='similarity', ascending=False).reset_index(drop=True)[:n]
    
    def get_recommendations(self, user: int, n: int=10):
        top_matches = self.top_matches(user, n)
        for cur_user in top_matches['user']:
            

In [118]:
user1 = 107720
user2 = 8659

In [122]:
cf = CollaborativeFiltering(df)
cf.top_matches(user=8659)

,user,similarity
90,13,0.348331
91,5,0.348331
92,100,0.356789
93,17,0.366025
94,23,0.400000
95,28,0.414214
96,85,0.500000
97,6,0.500000
98,26,0.500000
99,42,0.666667


In [105]:
common_movies_df = cf.common_movies(user1, user2)
# distance = cf.similarity_distance(common_movies_df)
# print(distance)

0.019693954869139264


In [111]:
df.userId.nunique()

265879

In [54]:
col = CollaborativeFiltering(df)
col.common_movies(user1=107720, user2=9998)

[480, 593]

In [48]:
def sim_pearson(critics_dict, person1, person2):
    sim_films = similar_films(critics_dict, person1, person2)
    if len(sim_films) < 2:
        return 0
    scores1 = []
    scores2 = []
    for film in sim_films:
        scores1.append(critics_dict[person1][film])
        scores2.append(critics_dict[person2][film])
    r = pearson_corr(scores1, scores2)
    if isnan(r):
        r = 0
    return r

## Задание 6

Используйте метод Term Frequency Inverse Document Frequency (TF-IDF), чтобы отфильтровать фильмы, похожие (используйте для этого косинусное расстояние) на те, которые пользователь высоко оценил. 

Оцените качество такой рекомендации.